In [23]:
#Importing package

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import glob
import statistics
import os
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

In [24]:
# prepare object points and image points for calibration

#Array to strore img points and obj point
objpoints = []
imgpoints = []

def calibrate():

    nx = 9 #Number of inside corners in x
    ny = 6 #Number of inside corners in y

    #Object Points
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Make a list of calibration images
    images = glob.glob('./camera_cal/calibration*.jpg')
    
    for fname in images:

        img = cv2.imread(fname)

        # Convert to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
        cv2.drawChessboardCorners(img, (nx, ny), corners, ret)

        # If found, update lists
        if ret == True:

            objpoints.append(objp)
            imgpoints.append(corners)

            # Draw and display the corners
            #cv2.drawChessboardCorners(img, (nx, ny), corners, ret)



In [25]:
#Function to undistort

def cal_undistort(img, objpoints, imgpoints):
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist


In [26]:
def sx_gradient_s_transform(img, s_thresh=(170, 255), sx_thresh=(50, 100)):
    
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    #color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255
    
    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
    return combined_binary


In [27]:
def perspective_transform(img):
    
    img_size = (img.shape[1],img.shape[0])
    
    offset = 200
    
    src = np.float32([[550,478],[732,478],[1053,685],[246,685]])

    dst = np.float32([[offset, offset], [img_size[0]-offset, offset], 
                                     [img_size[0]-offset, img_size[1]], 
                                     [offset, img_size[1]]])
    
    M = cv2.getPerspectiveTransform(src,dst) 
    warped = np.copy(img)
    warped = cv2.warpPerspective(warped,M,img_size, flags = cv2.INTER_LINEAR)

        
    return warped, src, dst




In [28]:
#Detect Lane Lines and Lane Boundaries


def find_lane_pixels(binary_warped):
    
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        ### Find the four below boundaries of the window ###
        win_xleft_low = leftx_current - margin  
        win_xleft_high = leftx_current + margin  
        win_xright_low = rightx_current - margin 
        win_xright_high = rightx_current + margin  
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        ### Identify the nonzero pixels in x and y within the window ###
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        ### If you found > minpix pixels, recenter next window ###
        ### (`right` or `leftx_current`) on their mean position ###
        
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))


    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img



In [68]:
#Search when you have a prior

def fit_poly(img, leftx, lefty, rightx, righty):
    ### Fit a second order polynomial to each with np.polyfit() ###
    
    global left_fit
    global right_fit
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0])
    ### Calc both polynomials using ploty, left_fit and right_fit ###
    
    try:
        left_fit = np.polyfit(lefty,leftx,2)
        right_fit = np.polyfit(righty,rightx,2)
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        #num_of_missed_frame = 0
    except:
        #print('The function failed to fit a line!')
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        pass
        
    return left_fitx, right_fitx, ploty

def search_around_poly(binary_warped):
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!
    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    ### Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = ((nonzerox > (left_fit[0]*nonzeroy**2 + left_fit[1]*nonzeroy + left_fit[2] - margin))
                    & (nonzerox < (left_fit[0]*nonzeroy**2 + left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*nonzeroy**2 + right_fit[1]*nonzeroy + right_fit[2] - margin))
                    & (nonzerox < (right_fit[0]*nonzeroy**2 + right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    if( (len(left_lane_inds) == 0) or (len(right_lane_inds) == 0)):
        num_of_missed_frame = num_of_missed_frame + 1
       
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_fitx, right_fitx, ploty = fit_poly(binary_warped, leftx, lefty, rightx, righty)
    
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    return result, left_fitx, right_fitx

In [69]:
def measure_curvature_real(leftx, rightx):
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/890 # meters per pixel in x dimension
    
    ploty = np.linspace(0, 719, num=720)   
    left_fit_cr = np.polyfit(ploty * ym_per_pix, leftx * xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty * ym_per_pix, rightx * xm_per_pix, 2)
    
    
    # Define y-value where we want radius of curvature
     # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = 719
    
    left_curverad = ((1+(2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**(3/2))/(2*left_fit_cr[0])
    
    ## Implement the calculation of the left line here
    right_curverad = ((1+(2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**(3/2))/(2*right_fit_cr[0])
    
    
    left_lane_Bottom = (left_fit_cr[0] * ((y_eval*ym_per_pix)**2)) + (left_fit_cr[1] * (y_eval*ym_per_pix)) + left_fit_cr[2]
    right_lane_Bottom = (right_fit_cr[0] * ((y_eval*ym_per_pix)**2)) + (right_fit_cr[1] * (y_eval*ym_per_pix)) + right_fit_cr[2]
    
#     mean_leftx = statistics.mean(left_fit_cr)
#     mean_lefty = statistics.mean(left_fit_cr)
#     centre_of_lane = (mean_leftx + mean_lefty)/2
#     centre_of_frame = 640 # Image size 1280/2
#     offset = (centre_of_frame - centre_of_lane) * xm_per_pix

    centre_of_lane = (left_lane_Bottom + right_lane_Bottom)/2
    centre_of_frame = 640 * xm_per_pix  # Image size 1280/2
    offset = (centre_of_frame - centre_of_lane)
    
    return left_curverad, right_curverad, offset
    

In [70]:
#Smoothning

def smooth(prev, curr, coeficient = 0.4):

    return curr*coeficient + prev*(1-coeficient)


In [71]:
left_fit = []
right_fit = []
num_of_missed_frame = 0
radius_of_curvature_previous = 0
offset_previous = 0
left_fit_previous = [0,0,0]
right_fit_previous = [0,0,0]

def process_image(image):
    
    global left_fit
    global right_fit
    global num_of_missed_frame
    global radius_of_curvature_previous
    global offset_previous
    global left_fit_previous
    global right_fit_previous
    
    if num_of_missed_frame == 5: #Reset if 5 frames are missed
        leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped_image)
        left_fit = np.polyfit(lefty, leftx, 2)
        left_fit_previous = left_fit
        right_fit_previous = right_fit
        right_fit = np.polyfit(righty, rightx, 2)
        lane_image, left_fitx, right_fitx = search_around_poly(binary_warped_image)
        num_of_missed_frame = 0
        
    if not imgpoints:
        calibrate()

    ploty = np.linspace(0, 719, num=720)    
        
    #image = mpimg.imread('./test_images/test6.jpg')
    undistort_image = cal_undistort(image, objpoints, imgpoints)
    combined_binary_image = sx_gradient_s_transform(undistort_image)
    binary_warped_image, src, dst = perspective_transform(combined_binary_image)
    
    if (len(left_fit) == 0) or (len(right_fit)==0):
        leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped_image)
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)
        left_fit_previous = left_fit
        right_fit_previous = right_fit
        lane_image, left_fitx, right_fitx = search_around_poly(binary_warped_image)
    else:
        lane_image, left_fitx, right_fitx = search_around_poly(binary_warped_image)
        
    
    warped = cv2.cvtColor(lane_image, cv2.COLOR_BGR2GRAY)

    left_curverad, right_curverad, offset = measure_curvature_real(left_fitx, right_fitx)
    
    radius_of_curvature_current = (left_curverad + right_curverad)/2
    radius_of_curvature = smooth(radius_of_curvature_previous,radius_of_curvature_current)
    radius_of_curvature_previous = radius_of_curvature
    
    offset = smooth(offset_previous,offset)
    offset_previous = offset
    
    left_fit[0]= smooth(left_fit_previous[0],left_fit[0])
    left_fit_previous[0] = left_fit[0]
    
    left_fit[1]= smooth(left_fit_previous[1],left_fit[1])
    left_fit_previous[1] = left_fit[1]
    
    left_fit[2]= smooth(left_fit_previous[2],left_fit[2])
    left_fit_previous[2] = left_fit[2]
    
    
    right_fit[0]= smooth(right_fit_previous[0],right_fit[0])
    right_fit_previous[0] = right_fit[0]
    
    right_fit[1]= smooth(right_fit_previous[1],right_fit[1])
    right_fit_previous[1] = right_fit[1]
    
    right_fit[2]= smooth(right_fit_previous[2],right_fit[2])
    right_fit_previous[2] = right_fit[2]
    
    
    #Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    Minv = cv2.getPerspectiveTransform(dst,src) 

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(undistort_image, 1, newwarp, 0.3, 0)

    cv2.putText(result,'RADIUS OF CURVATURE',(10,50),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),1)
    cv2.putText(result,str(round(radius_of_curvature, 2)),(450,50),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),1)
    cv2.putText(result,'OFFSET',(10,100),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),1)
    cv2.putText(result,str(round(offset, 2)),(190,100),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),1)
    
    return result


In [72]:
#Test Video

white_output = 'Output_Video/project_video.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
#clip1 = VideoFileClip("./project_video.mp4").subclip(0,3)
clip1 = VideoFileClip("./project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video Output_Video/project_video.mp4
[MoviePy] Writing video Output_Video/project_video.mp4


100%|█████████▉| 1260/1261 [27:21<00:01,  1.30s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: Output_Video/project_video.mp4 

CPU times: user 25min 3s, sys: 15.3 s, total: 25min 18s
Wall time: 27min 24s


In [1]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

NameError: name 'HTML' is not defined